In [1]:
import ipywidgets as widgets
from IPython.display import display

In [2]:
buyer_sliders = {
    key: widgets.IntSlider(description=key)
    for key in (
        'high', # $
        'buyer_time_price', # $/min
        'exp_low', # $
        'exp_seller_time_price', # $/min
        'buyer_exp_market', # $
        'buy_time_fungibility', # min
        )}

In [3]:
seller_sliders = {
    key: widgets.IntSlider(description=key)
    for key in [
        'low', # $
        'seller_time_price', # $/min
        'exp_high', # $
        'exp_buyer_time_price', # $/min
        'seller_exp_market', # $
        'sell_time_fungibility', # min
        ]}

In [22]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [37]:
# https://stackoverflow.com/questions/32000934/python-print-a-variables-name-and-value
import sys
def debug(expression):
    frame = sys._getframe(1)
    print(expression, ':\t', repr(eval(expression, frame.f_globals, frame.f_locals)))

In [56]:
def calculate(high, buyer_time_price, exp_low, exp_seller_time_price, buyer_exp_market, buy_time_fungibility, low, seller_time_price, exp_high, exp_buyer_time_price, seller_exp_market, sell_time_fungibility):
    TIME_CONSTANT = 1 # min
    
    buyer_null_util = max(0, high - buyer_exp_market - (buyer_time_price * buy_time_fungibility))
    debug('buyer_null_util') # $
    seller_null_util = max(low, seller_exp_market - low - (seller_time_price * sell_time_fungibility))
    debug('seller_null_util') # $
    seller_bid = [exp_high]
    debug('seller_bid[0]') # $
    buyer_bid = [exp_low]
    debug('buyer_bid[0]') # $
    if buyer_bid[0] > seller_bid[0]:
        final_price = mean([buyer_bid[0], seller_bid[0]])
        debug('final_price') # $
        return
    else:
        gap_between_opening_prices = seller_bid[0] - buyer_bid[0]
        debug('gap_between_opening_prices') # $
    
    buyer_acc_imm_util = high - seller_bid[0]
    debug('buyer_acc_imm_util') # $
    seller_acc_imm_util = buyer_bid[0] - low
    debug('seller_acc_imm_util') # $
    
    # buyer
    if (
            buyer_time_price > gap_between_opening_prices  # BAD COMPARISON ($/min to $)
            and buyer_acc_imm_util > buyer_null_util
            ):
        print("buyer accepts")
        buyer_accepts = True
    else:
        buyer_accepts = False
    
    # seller
    if (
            seller_time_price > gap_between_opening_prices  # BAD COMPARISON ($/min to $)
            and seller_acc_imm_util > seller_null_util
            ):
        print("seller accepts")
        seller_accepts = True
    else:
        seller_accepts = False
    
    if buyer_accepts and not seller_accepts:
        final_price = seller_bid[0]
        debug('final_price') # $
        return
    elif seller_accepts and not buyer_accepts:
        final_price = buyer_bid[0]
        debug('final_price') # $
        return
    elif buyer_accepts and seller_accepts:
        final_price = mean([buyer_bid[0], seller_bid[0]])
        debug('final_price') # $
        return
    
    # neither accepted
    
    for t in range(1, 8):
        buyer_bid.append(buyer_bid[t-1] + (exp_seller_time_price * TIME_CONSTANT))
        seller_bid.append(seller_bid[t-1] - (exp_buyer_time_price * TIME_CONSTANT))
        debug('buyer_bid[{}]'.format(t))
        debug('seller_bid[{}]'.format(t))
        # lazy completion
        if buyer_bid[t] > seller_bid[t]:
            final_price = mean([buyer_bid[t], seller_bid[t]])
            debug('final_price') # $
            return

In [57]:
buyer_box = widgets.VBox([widgets.Label('buyer:')] + list(buyer_sliders.values()))
seller_box = widgets.VBox([widgets.Label('seller:')] + list(seller_sliders.values()))
output_box = widgets.interactive_output(calculate, {k: v for k, v in dict(buyer_sliders, **seller_sliders).items()})

widgets.VBox([widgets.HBox([buyer_box, seller_box]), output_box])